In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import pandas as pd
import os


In [6]:
data_dir = "Dataset"

In [25]:

# Create a list to hold the image file paths and labels
image_paths = []
labels = []

# Map class labels to integers
class_mapping = {"Basal_Cell_Carcinoma": 1, "Melanoma": 2, "Normal_skin": 3}

# Iterate through the subfolders and collect image paths and labels
for class_name, class_label in class_mapping.items():
    class_dir = os.path.join(data_dir, class_name)
    for filename in os.listdir(class_dir):
        if filename.endswith(".jpg"):  
            image_paths.append(os.path.join(class_dir, filename))
            labels.append(class_label)

# Create a DataFrame to hold the image paths and labels
data = pd.DataFrame({"image_path": image_paths, "label": labels})

# Split the data into train, test, and validation sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, valid_data = train_test_split(train_data, test_size=0.2, random_state=42)


In [26]:
# Define data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    fill_mode="nearest"
)

# For validation and test data, only rescale the images
valid_datagen = ImageDataGenerator(rescale=1.0 / 255)


In [28]:
# Convert the labels to string format in your DataFrame
train_data["label"] = train_data["label"].astype(str)
valid_data["label"] = valid_data["label"].astype(str)
test_data["label"] = test_data["label"].astype(str)

# Update the data generators with class_mode set to "categorical"
train_generator = train_datagen.flow_from_dataframe(
    train_data,
    x_col="image_path",
    y_col="label",
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode="categorical"
)

valid_generator = valid_datagen.flow_from_dataframe(
    valid_data,
    x_col="image_path",
    y_col="label",
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode="categorical"
)

test_generator = valid_datagen.flow_from_dataframe(
    test_data,
    x_col="image_path",
    y_col="label",
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode="categorical"
)


Found 4214 validated image filenames belonging to 3 classes.
Found 1054 validated image filenames belonging to 3 classes.
Found 1317 validated image filenames belonging to 3 classes.


In [29]:
# Load the InceptionV3 model with pre-trained weights and exclude the top layer
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(3, activation='softmax')(x)  # 3 classes

# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the pre-trained layers to avoid updating their weights during training
for layer in base_model.layers:
    layer.trainable = False


In [30]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
epochs = 10  # You can increase this number for better results

history = model.fit(train_generator,
                    steps_per_epoch=len(train_data) // batch_size,
                    epochs=epochs,
                    validation_data=valid_generator,
                    validation_steps=len(valid_data) // batch_size)


In [33]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=len(test_data) // batch_size)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")


41/41 [==============================] - 106s 3s/step - loss: 0.0887 - accuracy: 0.9665
Test accuracy: 96.65%


In [35]:
# Save the model in the native Keras format
model.save("skin_cancer_inception_keras")


# Load the model in the native Keras format
loaded_model = keras.models.load_model("skin_cancer_inception_keras")



INFO:tensorflow:Assets written to: skin_cancer_inception_keras/assets


INFO:tensorflow:Assets written to: skin_cancer_inception_keras/assets
